## Define Area of Interest (AOI)

Use the map below to draw a rectangle defining your processing area. The maximum size is approximately 600x600 pixels at 10m resolution.

In [ ]:
import openeo
from extractor import load_input_WAC
from inference import inference_WAC


CRS = "EPSG:3035"
RESOLUTION = 10  # important; the resolution is implicitely tied to the CRS; so we need to use UTM based CRS here
PATCH_SIZE = 64
OVERLAP_SIZE = 0

SPATIAL_EXTENT = {
    'west': 300000,
    'south': 9800000,
    'east': 305000,  
    'north': 9805000,  
    'crs': CRS  
}

N_CLASSES = 21

TEMPORAL_EXTENT = ['2023-06-01', '2023-08-01'] 
MAX_CLOUD_COVER = 85


JOB_OPTIONS = {'driver-memory': '2000m',
 'driver-memoryOverhead': '2000m',
 'executor-memory': '3000m',
 'executor-memoryOverhead': '3000m',
 'python-memory': '8000m',
 'max-executors': 20,
 "udf-dependency-archives": [
        "https://s3.waw3-1.cloudferro.com/swift/v1/project_dependencies/onnx_dependencies_1.16.3.zip#onnx_deps",
        "https://s3.waw3-1.cloudferro.com/swift/v1/project_dependencies/FusionUNet_OilpalmModel_20250718131837.zip#onnx_models"
        ]
 }


In [4]:


connection = openeo.connect("https://openeo.dataspace.copernicus.eu/")
connection.authenticate_oidc()

new_band_names = ['mask_weights'] + [f"prob_class_{c}" for c in range(N_CLASSES)]
ROI_cube = load_input_WAC(connection, SPATIAL_EXTENT, TEMPORAL_EXTENT, MAX_CLOUD_COVER, RESOLUTION, CRS)
ROI_inference_cube = inference_WAC(ROI_cube, patch_size=PATCH_SIZE, overlap = OVERLAP_SIZE)
#ROI_inference_cube = ROI_inference_cube.linear_scale_range(0, 101, 0, 101)
ROI_inference_cube =ROI_inference_cube.rename_labels(dimension = 'bands', target = new_band_names)


ratio_cube = ROI_inference_cube.save_result(format = "netCDF")


job = connection.create_job(ratio_cube,
    additional=JOB_OPTIONS
)

job.start_and_wait()



Authenticated using refresh token.
0:00:00 Job 'j-2507181142564c2c914eccd469bc2c06': send 'start'
0:00:12 Job 'j-2507181142564c2c914eccd469bc2c06': queued (progress 0%)
0:00:17 Job 'j-2507181142564c2c914eccd469bc2c06': queued (progress 0%)
0:00:24 Job 'j-2507181142564c2c914eccd469bc2c06': queued (progress 0%)
0:00:32 Job 'j-2507181142564c2c914eccd469bc2c06': queued (progress 0%)
0:00:42 Job 'j-2507181142564c2c914eccd469bc2c06': queued (progress 0%)
0:00:54 Job 'j-2507181142564c2c914eccd469bc2c06': queued (progress 0%)
0:01:10 Job 'j-2507181142564c2c914eccd469bc2c06': queued (progress 0%)
0:01:29 Job 'j-2507181142564c2c914eccd469bc2c06': running (progress N/A)
0:01:53 Job 'j-2507181142564c2c914eccd469bc2c06': running (progress N/A)
0:02:23 Job 'j-2507181142564c2c914eccd469bc2c06': running (progress N/A)
0:03:01 Job 'j-2507181142564c2c914eccd469bc2c06': running (progress N/A)
0:03:47 Job 'j-2507181142564c2c914eccd469bc2c06': running (progress N/A)
0:04:46 Job 'j-2507181142564c2c914eccd46

<BatchJob job_id='j-2507181142564c2c914eccd469bc2c06'>

In [5]:
OUTPUT_DIR = "C:/Git_projects/WAC/production/prediction/sliding_window_normalized" + job.job_id
OUTPUT_FILE_ROI = f"{OUTPUT_DIR}"
job.get_results().download_files(OUTPUT_FILE_ROI)


[WindowsPath('C:/Git_projects/WAC/production/prediction/sliding_window_normalizedj-2506031012114f9f83d1bbd803b0d53a/openEO.nc'),
 WindowsPath('C:/Git_projects/WAC/production/prediction/sliding_window_normalizedj-2506031012114f9f83d1bbd803b0d53a/job-results.json')]

In [11]:
ROI_inference_cube = ROI_inference_cube.save_result(format = "netCDF")


job_ROI = connection.create_job(ROI_inference_cube,
    additional=JOB_OPTIONS
)

job_ROI.start_and_wait()


buffer_inference_cube = buffer_inference_cube.save_result(format = "netCDF")

job_buffer = connection.create_job(buffer_inference_cube,
    additional=JOB_OPTIONS
)

job_buffer.start_and_wait()

#TODO I believe the same partitioner is being used, which is not ideal as the artefacts will continue to overlap. Is this caused by the merge_cubes?


0:00:00 Job 'j-250603080142426893ff124a7609890a': send 'start'
0:00:13 Job 'j-250603080142426893ff124a7609890a': created (progress 0%)
0:00:18 Job 'j-250603080142426893ff124a7609890a': created (progress 0%)
0:00:25 Job 'j-250603080142426893ff124a7609890a': created (progress 0%)
0:00:33 Job 'j-250603080142426893ff124a7609890a': running (progress N/A)
0:00:43 Job 'j-250603080142426893ff124a7609890a': running (progress N/A)
0:00:55 Job 'j-250603080142426893ff124a7609890a': running (progress N/A)
0:01:11 Job 'j-250603080142426893ff124a7609890a': running (progress N/A)
0:01:30 Job 'j-250603080142426893ff124a7609890a': running (progress N/A)
0:01:54 Job 'j-250603080142426893ff124a7609890a': running (progress N/A)
0:02:24 Job 'j-250603080142426893ff124a7609890a': running (progress N/A)
0:03:01 Job 'j-250603080142426893ff124a7609890a': running (progress N/A)
0:03:48 Job 'j-250603080142426893ff124a7609890a': running (progress N/A)
0:04:47 Job 'j-250603080142426893ff124a7609890a': running (progr

<BatchJob job_id='j-2506030807304b9f9c7c0a45c080cc10'>

In [12]:
import xarray as xr 

OUTPUT_DIR = "C:/Git_projects/WAC/production/prediction/ROI_" + job_ROI.job_id
OUTPUT_FILE_ROI = f"{OUTPUT_DIR}"
job_ROI.get_results().download_files(OUTPUT_FILE_ROI)



[WindowsPath('C:/Git_projects/WAC/production/prediction/ROI_j-250603080142426893ff124a7609890a/openEO.nc'),
 WindowsPath('C:/Git_projects/WAC/production/prediction/ROI_j-250603080142426893ff124a7609890a/job-results.json')]

In [13]:
OUTPUT_DIR = "C:/Git_projects/WAC/production/prediction/buffer_" + job_buffer.job_id
OUTPUT_FILE_BUFFER = f"{OUTPUT_DIR}"
job_buffer.get_results().download_files(OUTPUT_FILE_BUFFER)

[WindowsPath('C:/Git_projects/WAC/production/prediction/buffer_j-2506030807304b9f9c7c0a45c080cc10/openEO.nc'),
 WindowsPath('C:/Git_projects/WAC/production/prediction/buffer_j-2506030807304b9f9c7c0a45c080cc10/job-results.json')]

In [13]:
import matplotlib.pyplot as plt
ds_buffer = xr.open_dataset("production/prediction/buffer_j-2506020939034c7e91306cda66ba0a9c/openEO.nc")
ds_ROI = xr.open_dataset("production/prediction/ROI_j-25060209311144b9ab8c6ecb5c30e973/openEO.nc")

band_name = 'prediction'  # or 'unknown_band_1'

for i, t in enumerate(ds_buffer['t']):
    fig, ax = plt.subplots(figsize=(5, 5))

    # Get the bands for this timestep
    band_shift = ds_buffer[band_name].sel(t=t)
    band_central = ds_ROI[band_name].sel(t=t)

    # Plot the first band (e.g., central) with a greyscale colormap
    band_central.plot(ax=ax, cmap='Greys', alpha=0.5, add_colorbar=False)

    # Overlay the second band (e.g., shift) with a different colormap or with contours
    band_shift.plot(ax=ax, cmap='Reds', alpha=0.5, add_colorbar=False)

    ax.set_title(f'{band_name} at {str(t.values)[:10]}')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

    plt.tight_layout()
    plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Git_projects\\WAC\\openEO\\inference\\production\\prediction\\buffer_j-2506020939034c7e91306cda66ba0a9c\\openEO.nc'